Данные взяты из ЛР_6 (их подготовка была в 6 лабе)

чем вдохновлялся:
1) Прикладное машинное обучение с помощью Scikit-Learn и TensorFlow:
концепции, инструменты и техники для создания интеллектуальных систем
2) https://habr.com/ru/companies/otus/articles/816667/

In [1]:
import pandas as pd
import numpy as np

In [1]:
!pip install tensorflow

In [51]:
X_train=pd.read_csv('/content/X_train.csv')
X_test=pd.read_csv('/content/X_test.csv')

y_train=pd.read_csv('/content/y_train.csv')
y_test=pd.read_csv('/content/y_test.csv')

In [32]:
import tensorflow as tf

In [ ]:
# X_train_tf = tf.constant(X_train.values, dtype=tf.float32)
# X_test_tf  = tf.constant(X_test.values,  dtype=tf.float32)

# y_train_tf = tf.constant(y_train.values, dtype=tf.float32)
# y_test_tf  = tf.constant(y_test.values,  dtype=tf.float32)

In [52]:
X = tf.constant(X_train.values, dtype=tf.float32)

y = tf.constant(y_train.values, dtype=tf.float32)

In [53]:
learning_rate = 0.005
epochs = 1000

In [54]:
n_samples = X.shape[0]

# Регрессионная модель

Функция для создания слоев

In [19]:
def neuron_layer(X, n_neurons, name, activation=None):
  with tf.name_scope(name):
    n_inputs = X.shape[1]
    stddev = 2 / np.sqrt(n_inputs + n_neurons)
    init = tf.random.truncated_normal((n_inputs, n_neurons), stddev=stddev)
    W = tf.Variable(init, name="kernel")
    b = tf.Variable(tf.zeros([n_neurons]), name="bias")
    Z = tf.matmul(X, W) + b
    if activation is not None:
      return Z, W, b, activation(Z)
    else:
      return Z, W, b, Z

def relu(Z):
    return tf.maximum(0.0, Z)

def relu_derivative(Z):
    return tf.cast(Z > 0, tf.float32)

## Первая модель с 2-мя скрытыми слоями и с функцией активации ReLU

In [34]:
n_inputs = 10
n_hidden_1 = 64
n_hidden_2 = 32
n_outputs = 1

In [35]:
Z1, W1, b1, A1 = neuron_layer(X, n_hidden_1, "hidden1", activation=None)
Z2, W2, b2, A2 = neuron_layer(tf.zeros([X.shape[0], n_hidden_1]), n_hidden_2, "hidden2", activation=None)
Z3, W3, b3, output = neuron_layer(tf.zeros([X.shape[0], n_hidden_2]), n_outputs, "output", activation=None)

Функция потерь MSE

In [36]:
def mse_loss(y_true, y_pred):
  return tf.reduce_mean(tf.square(y_true - y_pred))

**Обучение**

In [39]:
for epoch in range(epochs):
  Z1 = tf.matmul(X, W1) + b1
  A1 = relu(Z1)

  Z2 = tf.matmul(A1, W2) + b2
  A2 = relu(Z2)

  Z3 = tf.matmul(A2, W3) + b3
  output= Z3

  loss = mse_loss(y, output)

  dLoss_dOutput = 2 * (output - y) / n_samples

  dLoss_dW3 = tf.matmul(tf.transpose(A2), dLoss_dOutput)
  dLoss_db3 = tf.reduce_sum(dLoss_dOutput, axis=0, keepdims=True)

  dLoss_dA2 = tf.matmul(dLoss_dOutput, tf.transpose(W3))
  dLoss_dZ2 = dLoss_dA2 * relu_derivative(Z2)
  dLoss_dW2 = tf.matmul(tf.transpose(A1), dLoss_dZ2)
  dLoss_db2 = tf.reduce_sum(dLoss_dZ2, axis=0, keepdims=True)

  dLoss_dA1 = tf.matmul(dLoss_dZ2, tf.transpose(W2))
  dLoss_dZ1 = dLoss_dA1 * relu_derivative(Z1)
  dLoss_dW1 = tf.matmul(tf.transpose(X), dLoss_dZ1)
  dLoss_db1 = tf.reduce_sum(dLoss_dZ1, axis=0, keepdims=True)

  W3.assign_sub(learning_rate * dLoss_dW3)
  b3.assign_sub(learning_rate * tf.squeeze(dLoss_db3, axis=0))
  W2.assign_sub(learning_rate * dLoss_dW2)
  b2.assign_sub(learning_rate * tf.squeeze(dLoss_db2, axis=0))
  W1.assign_sub(learning_rate * dLoss_dW1)
  b1.assign_sub(learning_rate * tf.squeeze(dLoss_db1, axis=0))

  if epoch % 50 == 0:
    print(f"Epoch {epoch}, Loss: {loss.numpy()}")

Epoch 0, Loss: 3773.47802734375
Epoch 50, Loss: 198.64723205566406
Epoch 100, Loss: 184.4783172607422
Epoch 150, Loss: 179.2926788330078
Epoch 200, Loss: 173.9338836669922
Epoch 250, Loss: 168.58375549316406
Epoch 300, Loss: 164.48976135253906
Epoch 350, Loss: 161.01673889160156
Epoch 400, Loss: 157.88558959960938
Epoch 450, Loss: 155.43576049804688
Epoch 500, Loss: 153.1112060546875
Epoch 550, Loss: 151.34063720703125
Epoch 600, Loss: 149.54693603515625
Epoch 650, Loss: 148.1747589111328
Epoch 700, Loss: 146.6547393798828
Epoch 750, Loss: 145.54074096679688
Epoch 800, Loss: 144.27525329589844
Epoch 850, Loss: 143.2101287841797
Epoch 900, Loss: 142.04637145996094
Epoch 950, Loss: 140.65518188476562


## Вторая модель с 3-мя скрытыми слоями, а также с тангенсиальной функцией активации

In [46]:
n_hidden_1 = 256
n_hidden_2 = 64
n_hidden_3 = 32
n_outputs = 1

In [47]:
def tanh(Z):
    return tf.math.tanh(Z)

def tanh_derivative(Z):
    return 1 - tf.math.tanh(Z)**2

In [48]:
Z1, W1, b1, A1 = neuron_layer(X, n_hidden_1, "hidden1", activation=relu)
Z2, W2, b2, A2 = neuron_layer(tf.zeros([X.shape[0], n_hidden_1]), n_hidden_2, "hidden2", activation=tanh)
Z3, W3, b3, A3 = neuron_layer(tf.zeros([X.shape[0], n_hidden_2]), n_hidden_3, "hidden3", activation=tanh)
Z4, W4, b4, output = neuron_layer(tf.zeros([X.shape[0], n_hidden_3]), n_outputs, "output", activation=None)


**Обучение**

In [49]:
for epoch in range(epochs):

    Z1 = tf.matmul(X, W1) + b1
    A1 = relu(Z1)

    Z2 = tf.matmul(A1, W2) + b2
    A2 = tanh(Z2)

    Z3 = tf.matmul(A2, W3) + b3
    A3 = tanh(Z3)

    Z4 = tf.matmul(A3, W4) + b4
    output = Z4

    loss = mse_loss(y, output)

    dLoss_dOutput = 2 * (output - y) / n_samples

    dLoss_dW4 = tf.matmul(tf.transpose(A3), dLoss_dOutput)
    dLoss_db4 = tf.reduce_sum(dLoss_dOutput, axis=0, keepdims=True)

    dLoss_dA3 = tf.matmul(dLoss_dOutput, tf.transpose(W4))
    dLoss_dZ3 = dLoss_dA3 * tanh_derivative(Z3)
    dLoss_dW3 = tf.matmul(tf.transpose(A2), dLoss_dZ3)
    dLoss_db3 = tf.reduce_sum(dLoss_dZ3, axis=0, keepdims=True)

    dLoss_dA2 = tf.matmul(dLoss_dZ3, tf.transpose(W3))
    dLoss_dZ2 = dLoss_dA2 * tanh_derivative(Z2)
    dLoss_dW2 = tf.matmul(tf.transpose(A1), dLoss_dZ2)
    dLoss_db2 = tf.reduce_sum(dLoss_dZ2, axis=0, keepdims=True)

    dLoss_dA1 = tf.matmul(dLoss_dZ2, tf.transpose(W2))
    dLoss_dZ1 = dLoss_dA1 * relu_derivative(Z1)
    dLoss_dW1 = tf.matmul(tf.transpose(X), dLoss_dZ1)
    dLoss_db1 = tf.reduce_sum(dLoss_dZ1, axis=0, keepdims=True)

    W4.assign_sub(learning_rate * dLoss_dW4)
    b4.assign_sub(learning_rate * tf.squeeze(dLoss_db4, axis=0))
    W3.assign_sub(learning_rate * dLoss_dW3)
    b3.assign_sub(learning_rate * tf.squeeze(dLoss_db3, axis=0))
    W2.assign_sub(learning_rate * dLoss_dW2)
    b2.assign_sub(learning_rate * tf.squeeze(dLoss_db2, axis=0))
    W1.assign_sub(learning_rate * dLoss_dW1)
    b1.assign_sub(learning_rate * tf.squeeze(dLoss_db1, axis=0))

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

Epoch 0, Loss: 3876.35498046875
Epoch 50, Loss: 227.5876922607422
Epoch 100, Loss: 169.0169677734375
Epoch 150, Loss: 145.1420440673828
Epoch 200, Loss: 133.78860473632812
Epoch 250, Loss: 124.91207885742188
Epoch 300, Loss: 122.67548370361328
Epoch 350, Loss: 121.70854949951172
Epoch 400, Loss: 116.48747253417969
Epoch 450, Loss: 114.13533782958984
Epoch 500, Loss: 112.1841049194336
Epoch 550, Loss: 110.55619812011719
Epoch 600, Loss: 109.0429458618164
Epoch 650, Loss: 108.16889953613281
Epoch 700, Loss: 106.541748046875
Epoch 750, Loss: 105.1368637084961
Epoch 800, Loss: 103.68286895751953
Epoch 850, Loss: 102.15996551513672
Epoch 900, Loss: 100.84544372558594
Epoch 950, Loss: 101.54236602783203


## Третья модель с 4-мя скрытыми слоями, а также с тангенсиальной функцией активации

In [55]:
n_hidden_1 = 256
n_hidden_2 = 64
n_hidden_3 = 32
n_hidden_4 = 16
n_outputs = 1

In [56]:
Z1, W1, b1, A1 = neuron_layer(X, n_hidden_1, "hidden1", activation=relu)
Z2, W2, b2, A2 = neuron_layer(tf.zeros([X.shape[0], n_hidden_1]), n_hidden_2, "hidden2", activation=tanh)
Z3, W3, b3, A3 = neuron_layer(tf.zeros([X.shape[0], n_hidden_2]), n_hidden_3, "hidden3", activation=relu)
Z4, W4, b4, A4 = neuron_layer(tf.zeros([X.shape[0], n_hidden_3]), n_hidden_4, "hidden4", activation=tanh)
Z5, W5, b5, output = neuron_layer(tf.zeros([X.shape[0], n_hidden_4]), n_outputs, "output", activation=None)

In [57]:
for epoch in range(epochs):

    Z1 = tf.matmul(X, W1) + b1
    A1 = relu(Z1)

    Z2 = tf.matmul(A1, W2) + b2
    A2 = tanh(Z2)

    Z3 = tf.matmul(A2, W3) + b3
    A3 = relu(Z3)

    Z4 = tf.matmul(A3, W4) + b4
    A4 = tanh(Z4)

    Z5 = tf.matmul(A4, W5) + b5
    output = Z5

    loss = mse_loss(y, output)

    dLoss_dOutput = 2 * (output - y) / n_samples

    dLoss_dW5 = tf.matmul(tf.transpose(A4), dLoss_dOutput)
    dLoss_db5 = tf.reduce_sum(dLoss_dOutput, axis=0, keepdims=True)

    dLoss_dA4 = tf.matmul(dLoss_dOutput, tf.transpose(W5))
    dLoss_dZ4 = dLoss_dA4 * tanh_derivative(Z4)
    dLoss_dW4 = tf.matmul(tf.transpose(A3), dLoss_dZ4)
    dLoss_db4 = tf.reduce_sum(dLoss_dZ4, axis=0, keepdims=True)

    dLoss_dA3 = tf.matmul(dLoss_dZ4, tf.transpose(W4))
    dLoss_dZ3 = dLoss_dA3 * relu_derivative(Z3)
    dLoss_dW3 = tf.matmul(tf.transpose(A2), dLoss_dZ3)
    dLoss_db3 = tf.reduce_sum(dLoss_dZ3, axis=0, keepdims=True)

    dLoss_dA2 = tf.matmul(dLoss_dZ3, tf.transpose(W3))
    dLoss_dZ2 = dLoss_dA2 * tanh_derivative(Z2)
    dLoss_dW2 = tf.matmul(tf.transpose(A1), dLoss_dZ2)
    dLoss_db2 = tf.reduce_sum(dLoss_dZ2, axis=0, keepdims=True)

    dLoss_dA1 = tf.matmul(dLoss_dZ2, tf.transpose(W2))
    dLoss_dZ1 = dLoss_dA1 * relu_derivative(Z1)
    dLoss_dW1 = tf.matmul(tf.transpose(X), dLoss_dZ1)
    dLoss_db1 = tf.reduce_sum(dLoss_dZ1, axis=0, keepdims=True)

    W5.assign_sub(learning_rate * dLoss_dW5)
    b5.assign_sub(learning_rate * tf.squeeze(dLoss_db5, axis=0))
    W4.assign_sub(learning_rate * dLoss_dW4)
    b4.assign_sub(learning_rate * tf.squeeze(dLoss_db4, axis=0))
    W3.assign_sub(learning_rate * dLoss_dW3)
    b3.assign_sub(learning_rate * tf.squeeze(dLoss_db3, axis=0))
    W2.assign_sub(learning_rate * dLoss_dW2)
    b2.assign_sub(learning_rate * tf.squeeze(dLoss_db2, axis=0))
    W1.assign_sub(learning_rate * dLoss_dW1)
    b1.assign_sub(learning_rate * tf.squeeze(dLoss_db1, axis=0))

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

Epoch 0, Loss: 3887.554443359375
Epoch 50, Loss: 298.17840576171875
Epoch 100, Loss: 248.35147094726562
Epoch 150, Loss: 183.6991424560547
Epoch 200, Loss: 163.6563262939453
Epoch 250, Loss: 255.3255615234375
Epoch 300, Loss: 151.0913848876953
Epoch 350, Loss: 134.2249755859375
Epoch 400, Loss: 130.45274353027344
Epoch 450, Loss: 135.17880249023438
Epoch 500, Loss: 117.2361831665039
Epoch 550, Loss: 121.71372985839844
Epoch 600, Loss: 110.02201843261719
Epoch 650, Loss: 118.68744659423828
Epoch 700, Loss: 102.10443115234375
Epoch 750, Loss: 109.78971099853516
Epoch 800, Loss: 122.42039489746094
Epoch 850, Loss: 102.56536865234375
Epoch 900, Loss: 101.29076385498047
Epoch 950, Loss: 94.26465606689453


# Вывод


В работе была реализована простая полносвязная нейронная сеть для задачи регрессии полностью с нуля, без использования готовых слоёв, функций потерь и оптимизаторов. Все вычисления выполнены только через базовые операции TensorFlow.

**1. Экспериментальные архитектуры**

*Модель 1* — 2 скрытых слоя (64 → 32), ReLU–ReLU
Финальная ошибка: ~140.6

*Модель 2* — 3 скрытых слоя (256 → 64 → 32), ReLU–tanh–tanh
Финальная ошибка: ~101.5

*Модель 3* — 4 скрытых слоя (256 → 64 → 32 → 16), ReLU–tanh–ReLU–tanh
Финальная ошибка: ~94.2  
Это лучшая модель.

**2. Выбор функций активации**

- В начальных слоях использовалась ReLU, так как она устойчива к затуханию градиента и позволяет эффективно извлекать признаки.
- В более глубоких слоях использовалась tanh для стабилизации значений и повышения нелинейности.
- В выходном слое активация не использовалась, что правильно для регрессии.

**3. Выбор функции потерь**

В качестве функции потерь применена Mean Squared Error (MSE), так как задача является регрессией. MSE проста для ручной реализации градиента и чувствительна к большим ошибкам.

**4. Динамика обучения**

- Во всех моделях ошибка быстро снижалась в начале обучения.
- У более глубоких моделей наблюдались небольшие скачки ошибки, что связано скорее всего с использованием простого SGD.
- Самая глубокая модель показала наилучшую сходимость и минимальную ошибку.

**Итог**

Более глубокие архитектуры и комбинации ReLU и tanh улучшают качество модели. Лучшая сеть достигла ошибки около 94. Это хороший результат для модели, обученной полностью вручную. Выполнена реализация полного цикла обучения: прямой проход, обратное распространение ошибки, вычисление градиентов и обновление параметров.
